In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# import

In [3]:
# export
import torchvision

In [4]:
# export
from torch import nn

In [5]:
# export
import torch

In [6]:
import torchvision

In [7]:
from IPython.core import debugger as idb

# config

# create model

## functions

### flatten_grid_anchor

In [8]:
# export
def flatten_grid_anchor(x,k):
    '''
    将grid和anchor平铺。
    ssd头的卷积输出为 bs*nf*gx*gy，其中nf=num_anchor*num_pred，该函数将其整形为 bs*(gx*gy*num_anchor)*num_pred.
    ------------------------------
    参数：
    -- x：待整形的tensor，其维度为 bs*nf*gx*gy
    -- k：每个grid的anchor数
    ------------------------------
    返回值：
    -- 整形后的tensor，其维度为 bs*(gx*gy*k)*num_pred
    '''
    bs,nf,gx,gy = x.size()
#     x = x.permute(0,2,3,1).contiguous() # 横向（y）先变化
    x = x.permute(0,3,2,1).contiguous() # 纵向（x）先变化
    return x.view(bs,-1,nf//k)

### ssd_block

In [9]:
# export
class ssd_block(nn.Module):
    def __init__(self, k, nin, n_clas):
        '''
        ssd头模块，它根据某层的特征图给出bbox预测信息，该模块的输出包含4个部分：
        -- loc：bbox中心偏移，2个值
        -- conf：目标信心，1个值
        -- clas：目标类别，n_clas个值
        -- hw：bbox的h和w的缩放信息，2个值
        ----------------------------------------
        参数：
        -- k：每个grid的anchor数
        -- nin：输入特征图通道数
        -- n_clas：目标类别数
        '''
        super().__init__()
        self.k = k
        self.oconv_loc = nn.Conv2d(nin, 2*k, 3, padding=1) # bbox center
        self.oconv_conf = nn.Conv2d(nin, 1*k, 3, padding=1) # confidence
        self.oconv_clas = nn.Conv2d(nin, n_clas*k, 3, padding=1) # classification
        self.oconv_hw = nn.Conv2d(nin, 2*k, 3, padding=1) # bbox height and width
        
    def forward(self, x):
        return (flatten_grid_anchor(self.oconv_loc(x), self.k),
                flatten_grid_anchor(self.oconv_conf(x), self.k),
                flatten_grid_anchor(self.oconv_clas(x), self.k),
                flatten_grid_anchor(self.oconv_hw(x), self.k))

### ResNetIsh_SSD

In [10]:
# export
from torchvision.models.resnet import conv1x1
class ResNetIsh_SSD(nn.Module):
    def __init__(self, block, layers, chs, strides, pred_layerIds, num_anchors, pred_block, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        '''
        类名称的含义: ResNetIsh_SSD，其中Ish表示某某风格的，ResNetIsh表示resnet风格的，ResNetIsh_SSD表示resnet风格的骨架+SSD风格的预测头部。
        resnet风格指其使用的基本模块与构造逻辑与resnet一致。
        ---------------------------------------------------------------------------------------
        修改自torchvision.models.resent.ResNet源码
        ---------------------------------------------------------------------------------------
        新增参数：
        --chs:
            各layer的channel数，其元素个数应与layers一致。
        --strides:
            各layer的stride，其元素个数应与layers一致。
        --pred_layerIds:
            用于预测的特征图（下面称为“预测特征图”，不规范的称谓）的层序号，其与layers对应，例如layers=[2,2,2,2,2]，
            则pred_layerIds=[2,3,4]表示layers=[2,2,*2,*2,*2]中以*号标出的层的特征图作为预测特征图。
        --num_anchors:
            各预测特征图的每个grid的anchor数。一个list，元素个数应与pred_layerIds一致。
        --pred_block:
            定义预测头的模块
        ---------------------------------------------------------------------------------------
        对原有参数含义的调整：
        --replace_stride_with_dilation:
            源代码中其长度必须为3，改为其长度应与layers列表的长度一样。
        '''
        super(ResNetIsh_SSD, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False for _ in layers]
        if len(replace_stride_with_dilation) != len(layers):
            raise ValueError("replace_stride_with_dilation should be None "
                             "or it's elements should be same as layers, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.res_blocks = nn.ModuleList()
        for i,ch,layer,stride,dilate in zip(range(len(chs)),chs,layers,strides,replace_stride_with_dilation):
            self.res_blocks.append(self._make_layer(block, ch, layer, stride=stride, dilate=dilate))
        
        self.pred_layerIds = pred_layerIds
        self.pred_blocks = nn.ModuleList()
        for i in range(len(pred_layerIds)):
            k = num_anchors[i]
            ch_in = chs[pred_layerIds[i]]
            self.pred_blocks.append(pred_block(k, ch_in, num_classes))
        

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        outs = []
        for i in range(len(self.res_blocks)):
            x = self.res_blocks[i](x)
            if i in self.pred_layerIds:
                outs += [self.pred_blocks[i-self.pred_layerIds[0]](x)]

        return outs

    def forward(self, x):
        outs = self._forward_impl(x)
        
        locs,confs,clss,hws = [],[],[],[]
        for out in outs:
            locs += [out[0]]
            confs += [out[1]]
            clss += [out[2]]
            hws += [out[3]]
        
        return (torch.cat(locs,dim=1),
                torch.cat(confs,dim=1),
                torch.cat(clss,dim=1),
                torch.cat(hws,dim=1))

### ResNetIsh_1SSD

In [11]:
# export
class ResNetIsh_1SSD(nn.Module):
    def __init__(self, block, layers, chs, strides, pred_layerIds, num_anchors, neck_block, head_block, head_chin=256, num_classes=1000, 
                 zero_init_residual=False, groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        '''
        类名称的含义: ResNetIsh_SSD，其中Ish表示某某风格的，ResNetIsh表示resnet风格的，ResNetIsh_SSD表示resnet风格的骨架+SSD风格的预测头部。
        resnet风格指其使用的基本模块与构造逻辑与resnet一致。
        ---------------------------------------------------------------------------------------
        修改自torchvision.models.resent.ResNet源码
        ---------------------------------------------------------------------------------------
        新增参数：
        --chs:
            各layer的channel数，其元素个数应与layers一致。
        --strides:
            各layer的stride，其元素个数应与layers一致。
        --pred_layerIds:
            用于预测的特征图（下面称为“预测特征图”，不规范的称谓）的层序号，其与layers对应，例如layers=[2,2,2,2,2]，
            则pred_layerIds=[2,3,4]表示layers=[2,2,*2,*2,*2]中以*号标出的层的特征图作为预测特征图。
        --num_anchors:
            不同于ResNetIsh_SSD，本类要求num_anchors是一个整数，因为本类共用一个head。
        --neck_block:
            连接resnet特征图与预测头的模块
        --head_block:
            定义预测头的模块
        --head_chin:
            预测头的输入通道数
        ---------------------------------------------------------------------------------------
        对原有参数含义的调整：
        --replace_stride_with_dilation:
            源代码中其长度必须为3，改为其长度应与layers列表的长度一样。
        '''
        super(ResNetIsh_1SSD, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False for _ in layers]
        if len(replace_stride_with_dilation) != len(layers):
            raise ValueError("replace_stride_with_dilation should be None "
                             "or it's elements should be same as layers, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.res_blocks = nn.ModuleList()
        for i,ch,layer,stride,dilate in zip(range(len(chs)),chs,layers,strides,replace_stride_with_dilation):
            self.res_blocks.append(self._make_layer(block, ch, layer, stride=stride, dilate=dilate))
        
        self.pred_layerIds = pred_layerIds
        #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
#         self.pred_blocks = nn.ModuleList()
#         for i in range(len(pred_layerIds)):
#             k = num_anchors[i]
#             ch_in = chs[pred_layerIds[i]]
#             self.pred_blocks.append(pred_block(k, ch_in, num_classes))
        ##############################
        self.neck_blocks = nn.ModuleList()
        for i in range(len(pred_layerIds)):
            ch_in = chs[pred_layerIds[i]]
            self.neck_blocks.append(neck_block(ch_in,head_chin))
            
        self.head_block = head_block(num_anchors, head_chin, num_classes)
        #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
        

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        outs = []
        for i in range(len(self.res_blocks)):
            x = self.res_blocks[i](x)
            if i in self.pred_layerIds:
                #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
#                 outs += [self.neck_blocks[i-self.pred_layerIds[0]](x)]
                ################################
                neck_out = self.neck_blocks[i-self.pred_layerIds[0]](x)
                outs += [self.head_block(neck_out)]
                #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

        return outs

    def forward(self, x):
        outs = self._forward_impl(x)
        
        locs,confs,clss,hws = [],[],[],[]
        for out in outs:
            locs += [out[0]]
            confs += [out[1]]
            clss += [out[2]]
            hws += [out[3]]
        
        return (torch.cat(locs,dim=1),
                torch.cat(confs,dim=1),
                torch.cat(clss,dim=1),
                torch.cat(hws,dim=1))

### conv1x1_bn_relu

In [12]:
# export
class cnv1x1_bn_relu(nn.Module):
    def __init__(self, chin, chout):
        '''
        point-wise-conv + batchnorm + relu
        '''
        super().__init__()
        self.pwConv = nn.Conv2d(chin,chout,1,bias=False)
        self.bn = nn.BatchNorm2d(chout)
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        out = self.pwConv(x)
        out = self.bn(out)
        out = self.relu(out)
        return out

## test

In [14]:
# 构建数据
from ..exp.nb_databunch import *

data = get_databunch(data_root='./data/ds_20200428/')
x,y = data.one_batch(denorm=False)

num_classes = len(data.train_ds.y.classes)-1

### ResNetIsh_SSD

In [ ]:
# 构建模型
m = ResNetIsh_SSD(block=torchvision.models.resnet.BasicBlock,
                  layers=[2,2,2,2,2],
                  chs=[64,128,256,512,1024],
                  strides=[1,2,2,2,2],
                  pred_layerIds=[2,3,4],
                  num_anchors=[4,3,3],
                  pred_block=ssd_block,
                  num_classes=16)

In [ ]:
# 跑一个batch
pred = m(x)

In [ ]:
# 查看输出的形状
print(f'type(pred)={type(pred)}')
print(f'len(pred)={len(pred)}')

print('-----------------------')
for p in pred:
    print(p.shape)
    
print('-----------------------')
print(49*49*4+25*25*3+13*13*3)

### ResNetIsh_1SSD

In [ ]:
# 构建模型
m = ResNetIsh_1SSD(block=torchvision.models.resnet.BasicBlock,
                   layers=[2,2,2,2,2],
                   chs=[64,128,256,512,1024],
                   strides=[1,2,2,2,2],
                   pred_layerIds=[2,3,4],
                   num_anchors=4,
                   neck_block=cnv1x1_bn_relu,
                   head_chin=256,
                   head_block=ssd_block,
                   num_classes=16)

In [ ]:
# 跑一个batch
pred = m(x)

In [ ]:
# 查看输出的形状
print(f'type(pred)={type(pred)}')
print(f'len(pred)={len(pred)}')

print('-----------------------')
for p in pred:
    print(p.shape)
    
print('-----------------------')
print(49*49*4+25*25*4+13*13*4)

### ResNetIsh_1SSD_2

In [ ]:
# 构建模型
m = ResNetIsh_1SSD(block=torchvision.models.resnet.BasicBlock,
                   layers=[2,2,2,2],
                   chs=[64,128,256,512],
                   strides=[1,2,2,2],
                   pred_layerIds=[1,2,3],
                   num_anchors=6,
                   neck_block=cnv1x1_bn_relu,
                   head_chin=128,
                   head_block=ssd_block,
                   num_classes=16)

In [ ]:
# 跑一个batch
pred = m(x)

In [ ]:
# 查看输出的形状
print(f'type(pred)={type(pred)}')
print(f'len(pred)={len(pred)}')

print('-----------------------')
for p in pred:
    print(p.shape)
    
print('-----------------------')
print(99*99*6+49*49*6+25*25*6)

### resnet34_1ssd

In [15]:
# 构建模型
m = ResNetIsh_1SSD(block=torchvision.models.resnet.BasicBlock,
                   layers=[3,4,6,3],
                   chs=[64,128,256,512],
                   strides=[1,2,2,2],
                   pred_layerIds=[1,2,3],
                   num_anchors=6,
                   neck_block=cnv1x1_bn_relu,
                   head_chin=256,
                   head_block=ssd_block,
                   num_classes=16)

In [16]:
# 跑一个batch
pred = m(x)

In [18]:
# 查看输出的形状
print(f'type(pred)={type(pred)}')
print(f'len(pred)={len(pred)}')

print('-----------------------')
for p in pred:
    print(p.shape)
    
print('-----------------------')
print(97*97*6+49*49*6+25*25*6)

type(pred)=<class 'tuple'>
len(pred)=4
-----------------------
torch.Size([64, 74610, 2])
torch.Size([64, 74610, 1])
torch.Size([64, 74610, 16])
torch.Size([64, 74610, 2])
-----------------------
74610


## zip as function

In [ ]:
# export
# 仅为其它模块测试的调用方便
def get_resnet18_ssd():
    return ResNetIsh_SSD(block=torchvision.models.resnet.BasicBlock,
                         layers       =[ 2,   2,   2,   2,    2],
                         chs          =[64, 128, 256, 512, 1024],
                         strides      =[ 1,   2,   2,   2,    2],
                         pred_layerIds=[           2,   3,    4],
                         num_anchors  =[           4,   3,    3],
                         pred_block=ssd_block,
                         num_classes=16)

In [ ]:
# export
# 仅为其它模块测试的调用方便
def get_resnet18_1ssd():
    return ResNetIsh_1SSD(block=torchvision.models.resnet.BasicBlock,
                          layers       =[ 2,   2,   2,   2,    2],
                          chs          =[64, 128, 256, 512, 1024],
                          strides      =[ 1,   2,   2,   2,    2],
                          pred_layerIds=[           2,   3,    4],
                          num_anchors  =4,
                          neck_block=cnv1x1_bn_relu,
                          head_chin=256,
                          head_block=ssd_block,
                          num_classes=16)

In [ ]:
# export
def get_resnet18_1ssd_2():
    return ResNetIsh_1SSD(block=torchvision.models.resnet.BasicBlock,
                           layers=[2,2,2,2],
                           chs=[64,128,256,512],
                           strides=[1,2,2,2],
                           pred_layerIds=[1,2,3],
                           num_anchors=6,
                           neck_block=cnv1x1_bn_relu,
                           head_chin=128,
                           head_block=ssd_block,
                           num_classes=16)

In [19]:
# export
def get_resnet34_1ssd():
    return ResNetIsh_1SSD(block=torchvision.models.resnet.BasicBlock,
                           layers=[3,4,6,3],
                           chs=[64,128,256,512],
                           strides=[1,2,2,2],
                           pred_layerIds=[1,2,3],
                           num_anchors=6,
                           neck_block=cnv1x1_bn_relu,
                           head_chin=256,
                           head_block=ssd_block,
                           num_classes=16)

# export

In [20]:
!python notebook2script.py --fname 'resnet_ssd.ipynb' --outputDir './exp/'

Converted resnet_ssd.ipynb to exp/nb_resnet_ssd.py
